In [1]:
import tensorflow as tf

In [2]:
def conv(x, out_c, name):
    in_c = x.get_shape()[-1].value
    with tf.name_scope(name) as scope:
        weight = tf.Variable(tf.truncated_normal([3, 3, in_c, out_c], stddev=0.1), name="weights")
        bias = tf.Variable(tf.truncated_normal([out_c], stddev=0.1), name="bias")
    return tf.nn.relu(tf.nn.conv2d(x, weight, [1, 1, 1 ,1,], padding='SAME') + bias, name=scope)

In [3]:
def maxpool(x, name):
    return tf.nn.max_pool(x, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME', name=name)

In [4]:
def fc(x, out_c, name):
    in_c = x.get_shape()[-1].value
    with tf.name_scope(name) as scope:
        weight = tf.Variable(tf.truncated_normal([in_c, out_c], stddev=0.1), name="weights")
        bias = tf.Variable(tf.truncated_normal([out_c], stddev=0.1), name="bias")
    return tf.add(tf.matmul(x, weight), bias)

In [ ]:
#网络模型  input_size = (?, 224, 224, 3)
def model(images, dropout, n_classes):
    #conv1
    conv1_1 = conv(images, 64, 'conv1_1')
    conv1_2 = conv(conv1_1, 64, 'conv1_2')
    pool1 = maxpool(conv1_2, 'pool1')
    # size = (?, 112, 112, 64)
    
    #conv2
    conv2_1 = conv(pool1, 128, 'conv2_1')
    conv2_2 = conv(conv2_1, 128, 'conv2_2')
    pool2 = maxpool(conv2_2, 'pool2')
    # size = (?, 56, 56, 128)
    
    #conv3
    conv3_1 = conv(pool2, 256, 'conv3_1')
    conv3_2 = conv(conv3_1, 256, 'conv3_2')
    conv3_3 = conv(conv3_2, 256, 'conv3_3')
    pool3 = maxpool(conv3_3, 'pool3')
    # size = (?, 28, 28, 256)
    
    #conv4
    conv4_1 = conv(pool3, 512, 'conv4_1')
    conv4_2 = conv(conv4_1, 512, 'conv4_2')
    conv4_3 = conv(conv4_2, 512, 'conv4_3')
    pool4 = maxpool(conv4_3, 'pool4')
    # size = (?, 14, 14, 512)
    
    #conv5
    conv5_1 = conv(pool4, 512, 'conv5_1')
    conv5_2 = conv(conv5_1, 512, 'conv5_2')
    conv5_3 = conv(conv5_2, 512, 'conv5_3')
    pool5 = maxpool(conv5_3, 'pool5')
    # size = (?, 7, 7, 512)
    
    # fully connected layer
    flatten = tf.reshape(pool5, [-1, 7 * 7 * 512])
    fc_6 = tf.nn.relu(fc(flatten, 4096, 'fc_6'))
    dropout1 = tf.nn.dropout(fc_6, dropout)
    
    fc_7 = tf.nn.relu(fc(dropout1, 4096, 'fc_7'))
    dropout2 = tf.nn.dropout(fc_7, dropout)
    
    out = fc(dropout2, n_classes, 'out')
    return out